In [69]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotnine as pn 
import os
import wget

Download FBI crime data

In [70]:
# each xls sheet has that year and the previous year. Download 2010, 2012, 2014, 2016. 2018, 2019
urls = ['https://ucr.fbi.gov/crime-in-the-u.s/2010/crime-in-the-u.s.-2010/tables/10tbl04.xls/output.xls',
'https://ucr.fbi.gov/crime-in-the-u.s/2012/crime-in-the-u.s.-2012/tables/4tabledatadecoverviewpdf/table_4_crime_in_the_united_states_by_region_geographic_division_and_state_2011-2012.xls/output.xls',
'https://ucr.fbi.gov/crime-in-the-u.s/2014/crime-in-the-u.s.-2014/tables/table-4/table_4_crime_in_the_united_states_by_region_geographic_division_and_state_2013-2014.xls/output.xls',
'https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-2/table-2.xls/output.xls',
'https://ucr.fbi.gov/crime-in-the-u.s/2018/crime-in-the-u.s.-2018/tables/table-4/table-4.xls/output.xls',
'https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/tables/table-4/table-4.xls/output.xls']

# name of each xls sheet
Crime_datasets = ['2009_2010.xls', '2011_2012.xls', '2013_2014.xls', '2015_2016.xls', '2017_2018.xls', '2019.xls']
# get working dir and change back slash to forward slash
cwd = os.getcwd()
cwd = cwd.replace(os.sep, '/')

for i in range(0, len(Crime_datasets)):
  print(cwd + "/" +Crime_datasets[i])
  if os.path.exists(cwd + "/" + Crime_datasets[i]):
    print('valid')
  else:
    print('Download the FBI crime dataset for ' + Crime_datasets[i])
    print('Dataset is not longer at '+ urls[i])
      # requested = wget.download(urls[i], Crime_datasets[i])


c:/Users/David Ryan/Desktop/Dave/R_Projects/FBI_Crime/2009_2010.xls
valid
c:/Users/David Ryan/Desktop/Dave/R_Projects/FBI_Crime/2011_2012.xls
valid
c:/Users/David Ryan/Desktop/Dave/R_Projects/FBI_Crime/2013_2014.xls
valid
c:/Users/David Ryan/Desktop/Dave/R_Projects/FBI_Crime/2015_2016.xls
valid
c:/Users/David Ryan/Desktop/Dave/R_Projects/FBI_Crime/2017_2018.xls
valid
c:/Users/David Ryan/Desktop/Dave/R_Projects/FBI_Crime/2019.xls
valid


Merge FBI data across years and standardize column names

In [ ]:
for i in range(0,len(Crime_datasets)):
  # empty df
  Crime_raw = pd.DataFrame()
  #read in xls
  Crime_raw = pd.DataFrame(pd.read_excel(Crime_datasets[i], skiprows= 3))
  # remove empty columns, needs 10 NaNs to drop
  Crime_raw.dropna(axis = 1, thresh= 10, inplace=True)
  # rename col names for consistancy 
  ColNames = list(Crime_raw.columns)
  Col_fix = [ii for ii in ColNames if ii.startswith('Pop')]
  Crime_raw.rename(columns = {Col_fix[0] : 'Population'}, inplace=True)
  
  Col_fix = [ii for ii in ColNames if ii.startswith('Murder')]
  Crime_raw.rename(columns = {Col_fix[0] : 'Murder'}, inplace=True)
  
  Col_fix = [ii for ii in ColNames if ii.startswith('Violent')]
  Crime_raw.rename(columns = {Col_fix[0] : 'Violent_Crime'}, inplace=True)
  
  Col_fix = [ii for ii in ColNames if ii.endswith('(revised definition)4')]
  if len(Col_fix) > 0: 
    Crime_raw.rename(columns = {Col_fix[0] : 'Revised_Rape'}, inplace=True)
  
  Col_fix = [ii for ii in ColNames if ii.endswith('(legacy definition)5')]
  if len(Col_fix) > 0: 
    Crime_raw.rename(columns = {Col_fix[0] : 'Rape'}, inplace=True)
  
  Col_fix = [ii for ii in ColNames if ii.startswith('Forcible')]
  if len(Col_fix) > 0: 
    Crime_raw.rename(columns = {Col_fix[0] : 'Rape'}, inplace=True)

  ColNames = list(Crime_raw.columns)
  

  

  display(Crime_raw.head(15))


In [ ]:
# loop: find every column starts with Unnamed
#Crime_raw.columns.str.startswith('Unn')
Crime_raw.columns.get_indexer('Unn')
# Col_fix = [ii for ii in ColNames if ii.startswith('Unn')]
# print(ColNames.get_loc(Col_fix))

## From R: 
#   for (col.i in seq_along(col.names)) {
#     if (startsWith(col.names[col.i], prefix = '..')){ # rename cols that start with '..' to _Per100k'
#       col.names[col.i] = paste0(col.names[col.i -1], '_Per100k')
#       }
#     }




# combine col to left name with row 0

# rename colname 


#   #rename columns leading with '..#'
#   # find col with Name
#   col.names = colnames(Crime.raw)
#   for (col.i in seq_along(col.names)) {
#     if (startsWith(col.names[col.i], prefix = '..')){ # rename cols that start with '..' to _Per100k'
#       col.names[col.i] = paste0(col.names[col.i -1], '_Per100k')
#       }
#     }
#   setnames(Crime.raw, names(Crime.raw),  col.names) # rename 
  
#   # remove extra cols if any
#   rm.col=  !endsWith(colnames(Crime.raw), suffix = '_Per100k_Per100k') # extra cols will have extra per100k
#   Crime.raw = Crime.raw[,..rm.col]# remove cols
  
#   #remove cols of raw numbers, leave per100k for comparisons across states
#   rm.col = endsWith(colnames(Crime.raw), suffix = '_Per100k')
#   rm.col[1:3] = TRUE # keep first 3 rows of Area, year and population 
#   Crime.raw = Crime.raw[,..rm.col]
#   # Remove numbers  and comma from Area
#   Crime.raw$Area = gsub('[0-9]', '', Crime.raw$Area)
#   Crime.raw$Area = gsub(',', '', Crime.raw$Area)

#   # edit rows:
#   Crime.raw = Crime.raw[Year != 'Percent change']# remove percent change rows
  
#   # If Area name is NA, rename it to row above
#   for (ii in 1:Crime.raw[,.N]) {
#       if (is.na(Crime.raw[ii, 1])){
#       Crime.raw[ii, 1] = Crime.raw[ii-1, 1]
#     }
#   }
#     two.years = unique(Crime.raw$Year)
#   if (two.years[1] == '2018'){
#     Crime.raw = Crime.raw[Year == '2019']
#   }    

#   # change to numeric
#   Crime.raw[,2:ncol(Crime.raw)] = as.data.table(sapply(Crime.raw[,2:ncol(Crime.raw)], as.numeric))
  
#   # Round each col,
#     Crime.raw %>% mutate_if(is.numeric, ~round(., digit = 2))  
#   # Make List by year
#   # Year.crime[[two.years[1]]] = Crime.raw[Year == two.years[1]]
#   # Year.crime[[two.years[2]]] = Crime.raw[Year == two.years[2]]
  
#   if (i ==1){
#     Crime = Crime.raw
#   }else{
#     if(sum(colnames(Crime.raw) == 'Rape_Per100k') == 0){
#        Crime = merge.data.table(Crime, Crime.raw, by = colnames(Crime.raw), all = TRUE)
#     }else{
#     Crime = merge.data.table(Crime, Crime.raw, by = colnames(Crime), all = TRUE)
#     setdiff (colnames(Crime), colnames(Crime.raw))
#     setdiff (colnames(Crime.raw), colnames(Crime))
#     }
#   }
#  }

# rm(Crime.raw)
